In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# CSV dosyasını oku
df = pd.read_csv('vk5_drop.csv')

# 'Name' ve diğer istenmeyen sütunları çıkar
df = df.drop(columns=['name'])

# 'Label' sütununu hedef değişken (y) olarak ayarla
y = df['label']

# LabelEncoder ile etiketleri 0, 1, 2, 3 olacak şekilde dönüştür
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Geri kalan sütunları özellikler (X) olarak ayarla
X = df.drop(columns=['label'])

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForest modelini oluştur
model = RandomForestClassifier(random_state=42)

# Hiperparametre aralığını belirle
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV ile en iyi hiperparametreleri bul
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# En iyi parametreleri kullanarak RandomForest modelini oluştur ve eğit
best_params = grid_search.best_params_
model = RandomForestClassifier(**best_params, random_state=42)
model.fit(X_train, y_train)

# Test seti ile tahmin yap
y_pred = model.predict(X_test)

# Log loss hesapla
y_pred_proba = model.predict_proba(X_test)
validation_loss = log_loss(y_test, y_pred_proba)

# Modeli değerlendir
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Best Parameters: {best_params}')
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)
print(f'Validation Log Loss: {validation_loss}')
